In [4]:
import sys
import os
sys.path.insert(0,os.path.abspath('..'))
from spatial_graphs.AmiraSpatialGraph import AmiraSpatialGraph,MatchBarrels
from spatial_graphs.Landmarks import Landmarks
from spatial_graphs.Surfaces import Surface
from spatial_graphs.Vectors import Vectors
from spatial_graphs.Alignment import Alignment
from dask import compute,multiprocessing,delayed
import pathlib
import shutil
import glob
import pandas as pd
import vtk
from scipy.spatial import distance
import numpy as np
import SimpleITK as sitk
import itk

In [5]:
def add_z_pt_above_below(pt,z_offset,pt_list):
    above = [pt[0],pt[1],pt[2]+z_offset]
    below = [pt[0],pt[1],pt[2]-z_offset]
    pt_list.append(below)
    pt_list.append(above)
    pt_list.append(pt)
    return pt_list

In [6]:
def validate_pt(pt,im):
    if pt[0]<0:
        pt[0] = 0
    if pt[0]>im.GetSize()[0]:
        pt[0] = im.GetSize()[0]
    if pt[1] < 0:
        pt[1] = 0
    if pt[1] > im.GetSize()[1]:
        pt[1] = im.GetSize()[1]
        
    return pt

In [7]:
def get_min_max(pts):
    min_x = 9999
    max_x = 0
    min_y = 9999
    max_y = 0
    
    for pt in pts:
        if pt[0] < min_x:
            min_x = pt[0]
        if pt[0] > max_x:
            max_x = pt[0]
        if pt[1] < min_y:
            min_y = pt[1]
        if pt[1] > max_y:
            max_y = pt[1]
    return min_x,max_x,min_y,max_y

In [8]:
def get_real_foreground_im(bb_pts,masked_im):
    # get actual ROI as foreground
    pt_list=[]
    pt_list = add_z_pt_above_below(bb_pts[0],0.5,pt_list)
    pt_list = add_z_pt_above_below(bb_pts[1],0.5,pt_list)
    pt_list = add_z_pt_above_below(bb_pts[2],0.5,pt_list)
    pt_list = add_z_pt_above_below(bb_pts[3],0.5,pt_list)

    surf=Surface(pts=pt_list).create_delunay_surface_3d(return_hull=True,output_filename=output_path+'bla.vtk')

    np_im = sitk.GetArrayFromImage(masked_im)
    nonzeros = np.where(np_im>0)
    inds_pts = np.array([nonzeros[0],nonzeros[1],np.zeros_like(nonzeros[0])]).transpose()

    l = Landmarks(pts=inds_pts)
    selected = l.get_landmarks_within_given_surface(surf)
    print(selected)
    
    return selected

In [9]:
def get_bounding_plane(hull,z_coord,height,translation,rotation_angle,manual_tx_mat,im):
    
    intersectoin_plane = Surface(polydata=hull.get_intersection_plane(z_coord))
    #neg_mat = [-1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
    if intersectoin_plane.surface.GetNumberOfPoints() > 3:
        height = height.split('[')[1].split(']')[0]
        #intersectoin_plane.write_surface_mesh(output_path+'before.vtk')
        trans = [float(translation.split(' ')[0]), float(translation.split(' ')[1])]
        intersectoin_plane.apply_transformation([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, \
                                                 -trans[0], -trans[1]-float(height), -z_coord, 1])
        #intersectoin_plane.write_surface_mesh(output_path+'translated.vtk')
        #
        #intersectoin_plane.apply_transformation([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, -trans[0], -trans[1], -z_coord, 1])
        #intersectoin_plane.write_surface_mesh(output_path+'translation.vtk')
        theta = -float(rotation_angle)
        intersectoin_plane.apply_transformation([np.cos(theta), -np.sin(theta), 0, 1, np.sin(theta), np.cos(theta), 0, 1, \
                                                 0, 0, 0, 1, 0, 0, 0, 1])
        #intersectoin_plane.write_surface_mesh(output_path+'rotation.vtk')
        intersectoin_plane.apply_transformation(manual_tx_mat,inverse=True)
        #intersectoin_plane.write_surface_mesh(output_path+'manual.vtk')
#         intersectoin_plane_cube = Surface(polydata=intersectoin_plane.create_delunay_surface_3d(return_hull=True,\
#                                                              output_filename=output_path+'vS1_plane.vtk'))
        #intersectoin_plane.write_surface_mesh(output_path+'{}_sec_cutting_plane.vtk'.format(sec_num))
        bounds = intersectoin_plane.surface.GetBounds()
        pt_list = []
        
        vtkpts = intersectoin_plane.surface.GetPoints()
        #print(selected)
        #XY_RES = 1
         
#         pt1 = [int(vtkpts.GetPoint(0)[0]/XY_RES),int(vtkpts.GetPoint(0)[1]),0]
#         pt2 = [int(vtkpts.GetPoint(1)[0]/XY_RES),int(vtkpts.GetPoint(1)[1]),0]
#         pt3 = [int(vtkpts.GetPoint(2)[0]/XY_RES),int(vtkpts.GetPoint(2)[1]),0]
#         pt4 = [int(vtkpts.GetPoint(3)[0]/XY_RES),int(vtkpts.GetPoint(3)[1]),0]
        
        # get im inds within the BB of ROI
        pt1 = validate_pt([int(bounds[0]/XY_RES),int(bounds[2]/XY_RES),0],im)
        pt2 = validate_pt([int(bounds[1]/XY_RES),int(bounds[3]/XY_RES),0],im)
        #pt3 = validate_pt(pt3,im)
        #pt4 = validate_pt(pt4,im)
        
        return [pt1,pt2]
    else:
        return None

In [10]:
def mask_im_region_from_bb(np_im,bb_pts):
    #np_im = sitk.GetArrayFromImage(im)
    #np_im[:,:] = 0
    
    #min_x,max_x,min_y,max_y = get_min_max(bb_pts)
    #print(min_x,max_x,min_y,max_y)
    np_im[bb_pts[0][1]:bb_pts[1][1],bb_pts[0][0]:bb_pts[1][0]] = 255
    masked_im = sitk.GetImageFromArray(np_im)
    #sitk.WriteImage(masked_im,'{}_binary_masked.tif'.format(op_filename))
    
    return np_im

In [11]:
def mark_binary_im(masked_im):
    contour_im = sitk.BinaryContour(masked_im,backgroundValue=0,foregroundValue=255)
    dil = sitk.BinaryDilateImageFilter()
    dil.SetKernelRadius(10)
    dil.SetBackgroundValue(0)
    dil.SetForegroundValue(255)
    contour_dilated = dil.Execute(contour_im)
    marked_im = sitk.Or(im,contour_dilated)
    return marked_im
    

In [ ]:
animal_name = 'MG48'
image_name = 'MG48_3Day_coronal'

file_pattern = '*S*.am'
images_path = '/nas1/Data_aman/00_Rabies/{}/00_Images/00_Confocal/ch_01_stacks/'.format(image_name)
image_file_pattern = 'S*.tif'
#rabies_landmarks_path = '/nas1/Data_Mythreya/MotorCortexProject/V4/0_Inputs/SpatialGraphs/Section_Graphs/Rabies/{}/rabies/'.format(animal_name)
#neun_landmarks_path = '/nas1/Data_Mythreya/MotorCortexProject/V0/0_Inputs/Landmarks/Manual/Rabies/NeuN/{}/'.format(animal_name)

manual_transform_path = '/nas1/Data_aman/00_Rabies/FINAL/Mythreya/MG48_cortex/transformation/'
manual_tx_file = manual_transform_path +'manualalign.hx'

auto_transform_path = '/nas1/Data_aman/00_Rabies/FINAL/Mythreya/MG48_cortex/transformation/automatic transformation/'
auto_depth_file_pattern = 'Depth*'
auto_height_file_pattern = 'Height*'
auto_translation_file_pattern = 'Translation*'
auto_rotation_file_pattern = 'TurnAngle*'

#surface_path_vS1 = '/nas1/Data_Mythreya/MotorCortexProject/V9/vM1_Ref_Frame/Outputs/Surfaces/{}_s1_hull.vtk'.format(animal_name)
surface_path_vM1_lhs = '/nas1/Data_Mythreya/MotorCortexProject/V9/vM1_Ref_Frame/Outputs/vM1_Ref_Surfaces/{}_lhs_vM1.vtk'.format(animal_name)
surface_path_vM1_rhs = '/nas1/Data_Mythreya/MotorCortexProject/V9/vM1_Ref_Frame/Outputs/vM1_Ref_Surfaces/{}_rhs_vM1.vtk'.format(animal_name)

output_path = '/nas1/Data_Mythreya/MotorCortexProject/Images_For_NeuN_Count/{}/'.format(animal_name)

XY_RES = 0.868

In [ ]:
#sg_3d = AmiraSpatialGraph(glob.glob(path + file_pattern)[0],read_header_only=True)
#landmarks3d = Landmarks()
#landmarks3d_neun = Landmarks()
z_cood_offset = 50
#vs1_hull = Surface(surface_path_vS1)
#s1_cube = Surface(polydata=s1_hull.create_delunay_surface_3d(make_cube=True,return_hull=True,\
#                                                             output_filename=output_path+'vS1_Cube.vtk'))

hull_lhs = Surface(surface_path_vM1_lhs)
hull_rhs = Surface(surface_path_vM1_rhs)

neg_mat = [-1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
coroanl_tr_mat = [-1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1]

hull_rhs.apply_transformation(neg_mat,inverse=True)
hull_rhs.apply_transformation(coroanl_tr_mat,inverse=True)
hull_lhs.apply_transformation(coroanl_tr_mat,inverse=True)

hull = Surface(polydata=hull_lhs.surface)
hull.append(hull_rhs.surface)
hull.write_surface_mesh(output_path+'vM1_hull.vtk')

#vm1_cube = Surface(polydata=vm1_hull.create_delunay_surface_3d(make_cube=True,return_hull=True,\
#                                                               output_filename=output_path+'vM1_Cube.vtk'))
#write_cube(vm1_cube.surface,output_path+'vM1_Cube.vtk')

#Manual --> automatic(rotation--->translation--->height--->depth)
# read auto translations
with open(manual_tx_file,'r') as f:
    manual_tx_lines = f.readlines()
#print(manual_tx_lines)
with open(glob.glob(auto_transform_path+auto_depth_file_pattern)[0],'r') as f:
    auto_depth_lines = f.readlines()
with open(glob.glob(auto_transform_path+auto_height_file_pattern)[0],'r') as f:
    auto_height_lines = f.readlines()
with open(glob.glob(auto_transform_path+auto_translation_file_pattern)[0],'r') as f:
    auto_translation_lines = f.readlines()
with open(glob.glob(auto_transform_path+auto_rotation_file_pattern)[0],'r') as f:
    auto_rotation_lines = f.readlines()

for file in sorted(glob.glob(images_path + image_file_pattern)):
    #sg = AmiraSpatialGraph(file,axis_directions=[1,1,1])
    print(file)
    
    # read tx mat from the file
    sec_num = int(os.path.basename(file)[1:-7])
    #sec_num = 8
    print(sec_num)
    
    if sec_num > 246:
        continue
    
    str_to_compare = '"S{:03d}.am" setTransform '.format(sec_num)
    #print(str_to_compare)
    tx_mat = []
    tx_mat_np = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]).reshape(4,4)
    for line in manual_tx_lines:
        if line.startswith(str_to_compare):
            #print(line)
            #tx_mat = (line[len(str_to_compare):-1].split(' '))
            for num in (line[len(str_to_compare):-1].split(' ')):
                tx_mat.append(float(num))
            tx_mat_np = np.array(tx_mat).reshape(4,4)
            break
    
    print(tx_mat_np)
    print(auto_rotation_lines[sec_num-1])
    print(auto_translation_lines[sec_num-1])
    print(auto_height_lines[sec_num-1])
    print(auto_depth_lines[sec_num-1])
    
    
    #continue
    print(os.path.exists(glob.glob(images_path+'S{}_*.tif'.format(sec_num))[0]))
    if os.path.exists(glob.glob(images_path+'S{}_*.tif'.format(sec_num))[0]):
        print('rading image')
        im = sitk.ReadImage((glob.glob(images_path+'S{}_*.tif'.format(sec_num))[0]))
        npim = sitk.GetArrayFromImage(im)
        # get s1 and m1 bounding box for this plane
        #s1_bb_pts = get_bounding_plane(vs1_hull,(sec_num-1) * z_cood_offset,tx_mat_np,im)
        # undo the manual transformation
        
        m1_bb_pts_lhs = get_bounding_plane(hull_lhs,(sec_num-1) * z_cood_offset,auto_height_lines[sec_num-1],\
                                       auto_translation_lines[sec_num-1],auto_rotation_lines[sec_num-1],tx_mat,im )
        m1_bb_pts_rhs = get_bounding_plane(hull_rhs,(sec_num-1) * z_cood_offset,auto_height_lines[sec_num-1],\
                                       auto_translation_lines[sec_num-1],auto_rotation_lines[sec_num-1],tx_mat,im )
        #print(s1_bb_pts)
        print(m1_bb_pts)
        #Landmarks(pts=[s1_bb_pts[0]]).write_landmarks(output_path+'{}_lower.landmarksAscii'.format(sec_num))
        #Landmarks(pts=[s1_bb_pts[1]]).write_landmarks(output_path+'{}_upper.landmarksAscii'.format(sec_num))
#         if s1_bb_pts is not None :
            
#             npim = mask_im_region_from_bb(npim,s1_bb_pts,)#output_path+'S'+str(sec_num)+'_vS1')
            #pixel_inds = get_real_foreground_im(s1_bb_pts,s1_masked_im)
            #Landmarks(pts=pixel_inds).write_landmarks(output_path+'s1_im_inds')
            #sitk.WriteImage(cropped_im,output_path+'{}_01_cropped.tif'.format(sec_num))
            
        if m1_bb_pts_lhs is not None :
            npim[:] = 0
            npim = mask_im_region_from_bb(npim,m1_bb_pts_lhs,)#output_path+'S'+str(sec_num)+'_vM1')
            
        if m1_bb_pts_rhs is not None :
            #npim[:] = 0
            npim = mask_im_region_from_bb(npim,m1_bb_pts_rhs,)#output_path+'S'+str(sec_num)+'_vM1')
            #sitk.WriteImage(cropped_im,output_path+'{}_01_cropped.tif'.format(sec_num))
        
        #else:
        if m1_bb_pts_lhs is not None or m1_bb_pts_rhs is not None:
            sitk.WriteImage(sitk.GetImageFromArray(npim),output_path+'S{}_binary.tif'.format(sec_num))
            marked_im = mark_binary_im(sitk.GetImageFromArray(npim))
            sitk.WriteImage(marked_im,output_path+'S{}_boundary_marked.tif'.format(sec_num))
            
            masked_im = sitk.Mask(im,sitk.GetImageFromArray(npim))
            sitk.WriteImage(masked_im,output_path+'S{}_masked.tif'.format(sec_num))
    
    

# Apply coronal section transformations

In [1]:
animal_name = 'MG48'
image_name = 'MG48_cortex'

file_pattern = '*S*.am'
images_path = '/nas1/Data_aman/00_Rabies/{}/00_Images/00_Confocal/ch_01_stacks/'.format(image_name)
image_file_pattern = 'S*.tif'
#rabies_landmarks_path = '/nas1/Data_Mythreya/MotorCortexProject/V4/0_Inputs/SpatialGraphs/Section_Graphs/Rabies/{}/rabies/'.format(animal_name)
#neun_landmarks_path = '/nas1/Data_Mythreya/MotorCortexProject/V0/0_Inputs/Landmarks/Manual/Rabies/NeuN/{}/'.format(animal_name)

manual_transform_path = '/nas1/Data_aman/00_Rabies/FINAL/Mythreya/MG48_cortex/transformation/'
manual_tx_file = manual_transform_path +'manualalign.hx'

auto_transform_path = '/nas1/Data_aman/00_Rabies/FINAL/Mythreya/MG48_cortex/transformation/automatic transformation/'
auto_depth_file_pattern = 'Depth*'
auto_height_file_pattern = 'Height*'
auto_translation_file_pattern = 'Translation*'
auto_rotation_file_pattern = 'TurnAngle*'

#surface_path_vS1 = '/nas1/Data_Mythreya/MotorCortexProject/V9/vM1_Ref_Frame/Outputs/Surfaces/{}_s1_hull.vtk'.format(animal_name)
surface_path_vM1_lhs = '/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs/vM1_Ref_Surfaces/{}_lhs_vM1.vtk'.format(animal_name)
surface_path_vM1_rhs = '/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs/vM1_Ref_Surfaces/{}_rhs_vM1.vtk'.format(animal_name)

landmarks_path = '/nas1/Data_Mythreya/MotorCortexProject/NeuN_Counts/{}/'.format(animal_name)
output_path = '/nas1/Data_Mythreya/MotorCortexProject/NeuN_Counts/{}/'.format(animal_name)

XY_RES = 0.868

In [2]:
def apply_section_transformation(landmarks,z_coord,height,translation,rotation_angle,manual_tx_mat):
    
    #intersectoin_plane = Surface(polydata=hull.get_intersection_plane(z_coord))
    #neg_mat = [-1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
    if len(landmarks.pts) > 0:
        landmarks.apply_transformation(manual_tx_mat,inverse=False)
        theta = float(rotation_angle)
        landmarks.apply_transformation([np.cos(theta), -np.sin(theta), 0, 1, np.sin(theta), np.cos(theta), 0, 1, \
                                                 0, 0, 0, 1, 0, 0, 0, 1])
        
        height = height.split('[')[1].split(']')[0]
        trans = [float(translation.split(' ')[0]), float(translation.split(' ')[1])]
        landmarks.apply_transformation([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, \
                                                 trans[0], trans[1]+float(height), z_coord, 1])
        
        
        
    return landmarks

In [12]:
z_cood_offset = 50
#vs1_hull = Surface(surface_path_vS1)
#s1_cube = Surface(polydata=s1_hull.create_delunay_surface_3d(make_cube=True,return_hull=True,\
#                                                             output_filename=output_path+'vS1_Cube.vtk'))

hull_lhs = Surface(surface_path_vM1_lhs)
hull_rhs = Surface(surface_path_vM1_rhs)

neg_mat = [-1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
coroanl_tr_mat = [-1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1]

hull_rhs.apply_transformation(neg_mat,inverse=True)
hull_rhs.apply_transformation(coroanl_tr_mat,inverse=True)
hull_lhs.apply_transformation(coroanl_tr_mat,inverse=True)

hull = Surface(polydata=hull_lhs.surface)
hull.append(hull_rhs.surface)
hull.write_surface_mesh(output_path+'vM1_hull.vtk')

#vm1_cube = Surface(polydata=vm1_hull.create_delunay_surface_3d(make_cube=True,return_hull=True,\
#                                                               output_filename=output_path+'vM1_Cube.vtk'))
#write_cube(vm1_cube.surface,output_path+'vM1_Cube.vtk')

#Manual --> automatic(rotation--->translation--->height--->depth)
# read auto translations
with open(manual_tx_file,'r') as f:
    manual_tx_lines = f.readlines()
#print(manual_tx_lines)
with open(glob.glob(auto_transform_path+auto_depth_file_pattern)[0],'r') as f:
    auto_depth_lines = f.readlines()
with open(glob.glob(auto_transform_path+auto_height_file_pattern)[0],'r') as f:
    auto_height_lines = f.readlines()
with open(glob.glob(auto_transform_path+auto_translation_file_pattern)[0],'r') as f:
    auto_translation_lines = f.readlines()
with open(glob.glob(auto_transform_path+auto_rotation_file_pattern)[0],'r') as f:
    auto_rotation_lines = f.readlines()

aligned_landmarks = []
for file in sorted(glob.glob(images_path + image_file_pattern)):
    #sg = AmiraSpatialGraph(file,axis_directions=[1,1,1])
    #print(file)
    
    # read tx mat from the file
    sec_num = int(os.path.basename(file)[1:-7])
    #sec_num = 8
    #print(sec_num)
    
    if sec_num > 246:
        continue
    
    str_to_compare = '"S{:03d}.am" setTransform '.format(sec_num)
    #print(str_to_compare)
    tx_mat = []
    tx_mat_np = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]).reshape(4,4)
    for line in manual_tx_lines:
        if line.startswith(str_to_compare):
            #print(line)
            #tx_mat = (line[len(str_to_compare):-1].split(' '))
            for num in (line[len(str_to_compare):-1].split(' ')):
                tx_mat.append(float(num))
            tx_mat_np = np.array(tx_mat).reshape(4,4)
            break
    
#     print(tx_mat_np)
#     print(auto_rotation_lines[sec_num-1])
#     print(auto_translation_lines[sec_num-1])
#     print(auto_height_lines[sec_num-1])
#     print(auto_depth_lines[sec_num-1])
    
    
    #continue
    #print(os.path.exists(landmarks_path+'S_{}/landmarks.landmarkAscii'.format(sec_num)))
    if os.path.exists(landmarks_path+'S_{}/landmarks.landmarkAscii'.format(sec_num)):
        print((landmarks_path+'S_{}/landmarks.landmarkAscii'.format(sec_num)))
        landmarks = Landmarks(landmarks_path+'S_{}/landmarks.landmarkAscii'.format(sec_num))
#         im = sitk.ReadImage((glob.glob(images_path+'S{}_*.tif'.format(sec_num))[0]))
#         npim = sitk.GetArrayFromImage(im)
        # get s1 and m1 bounding box for this plane
        #s1_bb_pts = get_bounding_plane(vs1_hull,(sec_num-1) * z_cood_offset,tx_mat_np,im)
        # undo the manual transformation
        tr_landmarks = apply_section_transformation(landmarks,(sec_num-1) * z_cood_offset,auto_height_lines[sec_num-1],\
                                       auto_translation_lines[sec_num-1],auto_rotation_lines[sec_num-1],tx_mat)
        for pt in tr_landmarks.pts:
            aligned_landmarks.append(pt)

Landmarks(pts=aligned_landmarks).write_landmarks(output_path+animal_name+'_neun_landmarks_new.LandmarkAscii')

/nas1/Data_Mythreya/MotorCortexProject/NeuN_Counts/MG48/S_10/landmarks.landmarkAscii
/nas1/Data_Mythreya/MotorCortexProject/NeuN_Counts/MG48/S_11/landmarks.landmarkAscii
/nas1/Data_Mythreya/MotorCortexProject/NeuN_Counts/MG48/S_12/landmarks.landmarkAscii
/nas1/Data_Mythreya/MotorCortexProject/NeuN_Counts/MG48/S_13/landmarks.landmarkAscii
/nas1/Data_Mythreya/MotorCortexProject/NeuN_Counts/MG48/S_14/landmarks.landmarkAscii
/nas1/Data_Mythreya/MotorCortexProject/NeuN_Counts/MG48/S_15/landmarks.landmarkAscii
/nas1/Data_Mythreya/MotorCortexProject/NeuN_Counts/MG48/S_16/landmarks.landmarkAscii
/nas1/Data_Mythreya/MotorCortexProject/NeuN_Counts/MG48/S_17/landmarks.landmarkAscii
/nas1/Data_Mythreya/MotorCortexProject/NeuN_Counts/MG48/S_18/landmarks.landmarkAscii
/nas1/Data_Mythreya/MotorCortexProject/NeuN_Counts/MG48/S_19/landmarks.landmarkAscii
/nas1/Data_Mythreya/MotorCortexProject/NeuN_Counts/MG48/S_1/landmarks.landmarkAscii
/nas1/Data_Mythreya/MotorCortexProject/NeuN_Counts/MG48/S_20/landm

In [ ]:
nr_yr = 15
init = [25000,25000,50000,50000,100000,100000,150000,150000,200000,200000,\
        300000,300000,350000,350000,40000,0,0,0,0,0,\
        0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
final = 0
interest = 1.25
for i in range(35):
    saving = init[i]
    final = (final +  saving) * interest
final